###### import important library

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np

###### Read the books dataset and explore it

In [ ]:
user=pd.read_csv('BX-Users.csv',sep=',', encoding='latin-1')

In [ ]:
user.head()

In [ ]:
user.shape

In [ ]:
user.columns

In [ ]:
user.info()

###### Clean up NaN values

In [ ]:
user.isnull().sum()

In [ ]:
user=user.dropna()

In [ ]:
user.isnull().sum()

In [ ]:
user.shape

In [ ]:
book=pd.read_csv('BX-Books.csv',sep=',',encoding='Latin-1')

In [ ]:
book.head()

In [ ]:
book.shape

In [ ]:
book.columns

In [ ]:
book.info()

In [ ]:
book.isnull().sum()

In [ ]:
book=book.dropna()

In [ ]:
book.isnull().sum()

###### Read the data where ratings are given by users

In [ ]:
rating=pd.read_csv('BX-Book-Ratings.csv',sep=',',encoding='Latin-1',nrows=10000)

In [ ]:
rating.head()

In [ ]:
rating.shape

In [ ]:
rating.info()

In [ ]:
rating.columns

In [ ]:
rating.isnull().sum()

In [ ]:
rating.describe()

In [ ]:
recommendation=pd.merge(rating,book,on='isbn')
recommendation.head()

###### Take a quick look at the number of unique users and books

In [ ]:
n_users = recommendation.user_id.nunique()
n_books = recommendation.isbn.nunique()

In [ ]:
print('Num. of Users: '+ str(n_users))
print('Num of Books: '+str(n_books))

###### Convert ISBN variable to numeric type in order

In [ ]:
isbn = recommendation.isbn.unique()
print(" Length of isbn List:", len(isbn))
def isbn_numeric(isbn):
    index = np.where(isbn==isbn)
    return index[0][0]

###### Convert user_id variable to numeric type in order

In [ ]:
user_id=recommendation.user_id.unique()
print(" Length of user_id List:", len(user_id))
def user_id_numeric(user_id):
    index = np.where(user_id==user_id)
    return index[0][0]

###### Convert both user_id and isbn to ordered list i.e. from 0...n-1

In [ ]:
recommendation['user_id_order'] = recommendation['user_id'].apply(user_id_numeric)
recommendation['isbn_id_order'] = recommendation['isbn'].apply(isbn_numeric)
recommendation.head()

###### Re-index columns to build matrix

In [ ]:
new_col_order = ['user_id_order', 'isbn_id', 'rating', 'book_title', 'book_author','year_of_publication','publisher','isbn','user_id']
recommendation= recommendation.reindex(columns= new_col_order)
recommendation.head()

###### Data Visualization

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
publications = {}
for year in recommendation['year_of_publication']:
    if str(year) not in publications:
        publications[str(year)] = 0
    publications[str(year)] +=1

publications = {k:v for k, v in sorted(publications.items())}

fig = plt.figure(figsize =(55, 15))
plt.bar(list(publications.keys()),list(publications.values()), color = 'green')
plt.ylabel("Number of books published")
plt.xlabel("Year of Publication")
plt.title("Number of books published yearly")
plt.margins(x = 0)
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
sns.countplot(y="book_author", data=recommendation,order=recommendation['book_author'].value_counts().index[0:15])
plt.title("No of books by an author (Top 15)")
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
sns.countplot(y="publisher", data=recommendation,order=recommendation['publisher'].value_counts().index[0:15])
plt.title("No of books published by a publisher (Top 15)")
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(x="rating", data=recommendation)
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
data = recommendation[recommendation['rating'] != 0]
sns.countplot(x="rating", data=data)
plt.title("Explicit Ratings")
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
sns.countplot(y="book_title", data=recommendation, order=recommendation['book_title'].value_counts().index[0:15])
plt.title("Number of Ratings for a book (Top 15)")
plt.show()

###### Split your data into two sets (training and testing)

In [ ]:
recommendation.columns

In [ ]:
recommendation['rating'].value_counts()

In [ ]:
fs=['isbn', 'user_id']

In [ ]:
x=recommendation[fs]
x.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x['isbn'] = le.fit_transform(x['isbn'])

In [ ]:
x.info()

In [ ]:
y=recommendation.rating
y

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score

In [ ]:
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.25,random_state=0)

###### Make predictions based on user and item variables

In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

In [ ]:
loreg=LogisticRegression()

In [ ]:
loreg.fit(x_train,y_train)

In [ ]:
ypred=loreg.predict(x_test)

In [ ]:
pd.DataFrame(confusion_matrix(y_test,ypred))

In [ ]:
print(classification_report(y_test,ypred))

In [ ]:
accuracy_score(y_test,ypred)*100

In [ ]:
# Decision Tree 
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtc=DecisionTreeClassifier()

In [ ]:
dtc.fit(x_train,y_train)

In [ ]:
ypred1=dtc.predict(x_test)

In [ ]:
pd.DataFrame(confusion_matrix(y_test,ypred1))

In [ ]:
print(classification_report(y_test,ypred1))

In [ ]:
accuracy_score(y_test,ypred1)*100

In [ ]:
# Ramdom Forest
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc=RandomForestClassifier()

In [ ]:
rfc.fit(x_train,y_train)

In [ ]:
ypred2=rfc.predict(x_test)

In [ ]:
pd.DataFrame(confusion_matrix(y_test,ypred2))

In [ ]:
print(classification_report(y_test,ypred2))

In [ ]:
accuracy_score(y_test,ypred2)*100

In [ ]:
#KNN
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knc=KNeighborsClassifier(n_neighbors=5)

In [ ]:
knc.fit(x_train,y_train)

In [ ]:
ypred5=knc.predict(x_test)

In [ ]:
pd.DataFrame(confusion_matrix(y_test,ypred5))

In [ ]:
print(classification_report(y_test,ypred5))

In [ ]:
accuracy_score(y_test,ypred5)*100

In [ ]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB

In [ ]:
nbg=GaussianNB()

In [ ]:
nbg.fit(x_train,y_train)

In [ ]:
ypred6=nbg.predict(x_test)

In [ ]:
pd.DataFrame(confusion_matrix(y_test,ypred6))

In [ ]:
print(classification_report(y_test,ypred6))

In [ ]:
accuracy_score(y_test,ypred6)*100

###### Use RMSE to evaluate the predictions

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

In [ ]:
print("mean squared error: {}" .format(mean_squared_error(y_test,ypred)))#Logistic
print("R square: {}".format(r2_score(y_test,ypred)))
print("RMSE: {}".format(sqrt(mse)))

In [ ]:
print("mean squared error: {}" .format(mean_squared_error(y_test,ypred1)))#decision
print("R square: {}".format(r2_score(y_test,ypred1)))
print("RMSE: {}".format(sqrt(mse)))

In [ ]:
print("mean squared error: {}" .format(mean_squared_error(y_test,ypred2)))#random forest
print("R square: {}".format(r2_score(y_test,ypred2)))
print("RMSE: {}".format(sqrt(mse)))

In [ ]:
print("mean squared error: {}" .format(mean_squared_error(y_test,ypred5))) #KNN
print("R square: {}".format(r2_score(y_test,ypred5)))
print("RMSE: {}".format(sqrt(mse)))

In [ ]:
print("mean squared error: {}" .format(mean_squared_error(y_test,ypred6)))# Naive bayes
print("R square: {}".format(r2_score(y_test,ypred6)))
print("RMSE: {}".format(sqrt(mse)))